In [0]:
import argparse
import os
import numpy as np
import math
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
from google.colab import drive
drive.mount("/content/drive")


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        print("***********",latent_dim)
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )
        self.avgpool =  nn.AdaptiveAvgPool2d(1)

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        print("----",int(np.prod(img_shape)))

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)

        return validity

def define_dataset():
	root_dir = 'drive/My Drive/SAMPLE/faces94'
	files = glob(f"{root_dir}/**/**/*.jpg")

	lab = {
	    "female": 0,
	    "male":1,
	    "malestaff": 2 
	}
	return files,lab

epochs = 20
batch_size = 32
learning_rate = 0.001
img_shape = (3, 200, 180)
interval = 100
b1 = 0.5
b2 = 0.999
latent_dim = 200

os.makedirs("images", exist_ok=True)
files,lab=define_dataset()
adversarial_loss = torch.nn.BCELoss()

generator = Generator()
discriminator = Discriminator()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(b1, b2))

Tensor = torch.FloatTensor



transform_loader = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

all_imgs = torch.stack([transform_loader(Image.open(x)) for x in files])
all_label = torch.tensor([lab[x.split('/')[-3]] for x in files])

train_idx, test_idx = train_test_split(range(len(all_imgs)), test_size=0.2, random_state=102)

train_img = all_imgs[train_idx]
train_label = all_label[train_idx]

test_img = all_imgs[test_idx]
test_label = all_label[test_idx]

train_data = TensorDataset(train_img, train_label)
test_data = TensorDataset(test_img, test_label)

train_samp = RandomSampler(train_data)
test_samp = SequentialSampler(test_data)

train_loader = DataLoader(train_data, sampler=train_samp, batch_size=32)
test_loader = DataLoader(test_data, sampler=test_samp, batch_size=32)

for epoch in range(epochs):
    for i, (imgs,_) in enumerate(train_loader):
      
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        real_imgs = Variable(imgs.type(Tensor))
        
        optimizer_G.zero_grad()

       
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        optimizer_D.zero_grad()

        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, epochs, i, len(train_loader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(train_loader) + i
        if batches_done % interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
*********** 200
---- 108000
